In [13]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple

In [14]:
# Load configuration from config.json
with open('../models/config.json', 'r') as f:
    config = json.load(f)

postprocess = config['postprocessing']

# Output and input paths from configuration
OUTPUT_PATH = postprocess['output_path']
INPUT_PATH_S = postprocess['input_path_s']
INPUT_PATH_D = postprocess['input_path_d']

# New Input paths
INPUT_PATH_WS = postprocess['input_path_ws']
INPUT_PATH_HD = postprocess['input_path_hd']
INPUT_PATH_D_OBJ = postprocess['input_path_d_obj']
INPUT_PATH_S_OBJ = postprocess['input_path_s_obj']
INPUT_PATH_D_OBJ_ACT = postprocess['input_path_d_obj_act']


# Display the loaded paths
print("Loaded paths:")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"INPUT_PATH_S: {INPUT_PATH_S}")
print(f"INPUT_PATH_D: {INPUT_PATH_D}")
print(f"INPUT_PATH_WS: {INPUT_PATH_WS}")
print(f"INPUT_PATH_HD: {INPUT_PATH_HD}")

special_case = 'USE_EXT_COST_10'


Loaded paths:
OUTPUT_PATH: ../data/postprocessing/
INPUT_PATH_S: ../data/output/stochastic/timeseries/
INPUT_PATH_D: ../data/output/deterministic/timeseries/
INPUT_PATH_WS: ../data/output/deterministic/cases/scenarios/
INPUT_PATH_HD: ../data/input/demands/


## Filename Generations Functions

In [15]:
def get_ws_solution_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the Wait and See solution objective values CSV file.
    """
    filename = f"d_scenarios_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_probabilities_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the probabilities JSON file.
    """
    filename = f"reduced_heat_demand_scenarios_{start_date}_to_{end_date}_{period}.json"
    return filename

def get_weighted_deterministic_ts_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted time series CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_ts.csv"
    return filename


def get_stochastic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the stochastic objective values CSV file.
    """
    filename = f"s_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_stochastic_objective_values_special_filename(start_date: str, end_date: str, period: str, special_case: str) -> str:
    """
    Generate the filename for the stochastic objective values CSV file.
    """
    filename = f"s_{start_date}_to_{end_date}_{period}_{special_case}_obj.csv"
    return filename

def get_actual_deterministic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the actual deterministic objective values CSV file.
    """
    filename = f"d_actual_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_deterministic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the actual deterministic objective values CSV file.
    """
    filename = f"d_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename


## Data Loading Functions

In [16]:
def load_ws_solution_data(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the Wait and See solution objective values for a specific date.
    """
    filename = get_ws_solution_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded Wait and See solution data from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_probabilities(date_str: str, period: str, input_path: str) -> Dict[str, float]:
    """
    Load probabilities from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: Probability}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded probabilities from {filepath}")
        # Extract probabilities
        probabilities = {}
        for scenario_name, scenario_data in data.items():
            probabilities[scenario_name] = scenario_data.get('Probability', 0)
        return probabilities
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    
def load_weighted_deterministic_ts(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted time series data for a specific date.
    """
    filename = get_weighted_deterministic_ts_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic weighted time series data from {filepath}")
        # Display the first few rows
        display(data.head())
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_scenario_heat_demands(date_str: str, period: str, input_path: str) -> Dict[str, pd.Series]:
    """
    Load scenario heat demands from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: pd.Series of heat demands}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded scenario heat demands from {filepath}")
        # For each scenario, create a pandas Series with time steps and heat demands
        scenario_heat_demands = {}
        for scenario_name, scenario_data in data.items():
            # Remove 'Probability' key and adjust time steps to zero-based indexing
            heat_demand_data = {int(k) - 1: v for k, v in scenario_data.items() if k != 'Probability'}
            # Convert to Series and sort by time step
            heat_demand_series = pd.Series(heat_demand_data).sort_index()
            scenario_heat_demands[scenario_name] = heat_demand_series
        return scenario_heat_demands
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    

def get_weighted_deterministic_objective_value_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted objective value CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def load_weighted_deterministic_objective_value(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted objective value for a specific date.
    """
    filename = get_weighted_deterministic_objective_value_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic objective value from {filepath}")
        # Display the data
        display(data)
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    
def load_stochastic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the stochastic objective values for a specific date.
    """
    filename = get_stochastic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded stochastic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()
    
def load_stochastic_objective_special_values(date_str: str, period: str, input_path: str, special_case: str) -> pd.DataFrame:
    """
    Load the stochastic objective values for a specific date.
    """
    filename = get_stochastic_objective_values_special_filename(date_str, date_str, period, special_case)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded stochastic objective special case values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_actual_deterministic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the actual deterministic objective values for a specific date.
    """
    filename = get_actual_deterministic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded actual deterministic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()
    
def load_deterministic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the actual deterministic objective values for a specific date.
    """
    filename = get_deterministic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded actual deterministic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()




# Calculations

In [17]:
# List of dates to process
dates_to_process = [
    datetime(2023, 3, 3),
    datetime(2023, 3, 16),
    datetime(2023, 4, 3),
    datetime(2023, 4, 9),
    datetime(2023, 5, 14),
    datetime(2023, 5, 15),
    datetime(2023, 6, 19),
    datetime(2023, 6, 22),
    datetime(2023, 6, 26),
    datetime(2023, 7, 9),
    datetime(2023, 7, 10),
    # Add mo
]

# Period for the analysis
period = 'day'


## Compute weighted Wait-and-See Solution

In [18]:
# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")
    
    # Load Wait and See solution data
    ws_data = load_ws_solution_data(date_str, period, INPUT_PATH_WS)
    if ws_data.empty:
        print(f"No Wait and See solution data available for date {date_str}.")
        continue
    
    # Display the loaded Wait and See solution data
    print("Wait and See Objective Values:")
    display(ws_data)
    
    # Load probabilities
    probabilities = load_probabilities(date_str, period, INPUT_PATH_HD)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        continue
    
    # Display the probabilities
    print("Probabilities:")
    probabilities_df = pd.DataFrame.from_dict(probabilities, orient='index', columns=['Probability'])
    display(probabilities_df)
    
    # Merge ws_data with probabilities
    ws_data['Probability'] = ws_data['Scenario'].map(probabilities)
    
    # Check for missing probabilities
    if ws_data['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(ws_data[ws_data['Probability'].isnull()])
        continue
    
    # Compute weighted objective value
    ws_data['WeightedObjective'] = ws_data['ObjectiveValue'] * ws_data['Probability']
    
    # Print intermediate calculations
    print("Intermediate Calculations:")
    display(ws_data)
    
    # Sum the weighted objectives
    weighted_objective_value = ws_data['WeightedObjective'].sum()
    
    # Store the result
    results.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Weighted Objective Value': weighted_objective_value
    })



Processing date: 20230303
Loaded Wait and See solution data from ../data/output/deterministic/cases/scenarios/d_scenarios_20230303_to_20230303_day_obj.csv
Wait and See Objective Values:


,Scenario,ObjectiveValue
0,Scenario1,-313.272646
1,Scenario2,-303.277273
2,Scenario3,-312.223968
3,Scenario4,-307.834008
4,Scenario5,-309.032745
5,Scenario6,-309.827177
6,Scenario7,-309.039271
7,Scenario8,-305.915359
8,Scenario9,-313.295285
9,Scenario10,-307.182671


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Probabilities:


,Probability
Scenario1,0.213933
Scenario2,0.085874
Scenario3,0.060216
Scenario4,0.115567
Scenario5,0.098109
Scenario6,0.064584
Scenario7,0.078224
Scenario8,0.061210
Scenario9,0.061475
Scenario10,0.160809


Intermediate Calculations:


...

,Scenario,ObjectiveValue
0,Scenario1,-114.493744
1,Scenario2,-96.786299
2,Scenario3,-144.461455
3,Scenario4,-120.222212
4,Scenario5,-123.113194
5,Scenario6,-125.500130
6,Scenario7,-134.425613
7,Scenario8,-138.759346
8,Scenario9,-109.955296
9,Scenario10,-117.287197


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Probabilities:


,Probability
Scenario1,0.157475
Scenario2,0.055415
Scenario3,0.134182
Scenario4,0.115740
Scenario5,0.069091
Scenario6,0.091832
Scenario7,0.110580
Scenario8,0.066341
Scenario9,0.130086
Scenario10,0.069256


Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-114.493744,0.157475,-18.029946
1,Scenario2,-96.786299,0.055415,-5.363443
2,Scenario3,-144.461455,0.134182,-19.384154
3,Scenario4,-120.222212,0.115740,-13.914528
4,Scenario5,-123.113194,0.069091,-8.506043
5,Scenario6,-125.500130,0.091832,-11.524930
6,Scenario7,-134.425613,0.110580,-14.864793
7,Scenario8,-138.759346,0.066341,-9.205478
8,Scenario9,-109.955296,0.130086,-14.303639
9,Scenario10,-117.287197,0.069256,-8.122897


#### Final Results Wait-and-See Solution

In [19]:
# Create a DataFrame of results
results_df = pd.DataFrame(results)
results_df['Weighted Objective Value'] = results_df['Weighted Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df)



Final Results:


,Date,Weighted Objective Value
0,2023-03-03,-309.324721
1,2023-03-16,-339.637840
2,2023-04-03,-358.514306
3,2023-04-09,-370.313316
4,2023-05-14,-164.013808
5,2023-05-15,-160.006004
6,2023-06-19,-117.059143
7,2023-06-22,-113.438573
8,2023-06-26,-112.173895
9,2023-07-09,-125.163018


## Compute Heat Demand Difference for the calculation of EEV

In [20]:
def compute_costs_from_differences(date_str: str, period: str, input_path_det_ts: str, input_path_hd: str):
    """
    Compute the differences and costs between deterministic and scenario heat demands for a specific date.
    Returns:
        - total_costs_df: DataFrame containing total costs per scenario.
        - weighted_total_cost: The total weighted cost for the date.
    """
    # Load deterministic weighted time series data
    det_ts_data = load_weighted_deterministic_ts(date_str, period, input_path_det_ts)
    if det_ts_data.empty:
        print(f"No deterministic weighted time series data available for date {date_str}.")
        return None, None

    # Extract the deterministic heat demand
    det_heat_demand = det_ts_data['heat_demand']
    det_heat_demand.name = 'deterministic_heat_demand'

    # Load scenario heat demands
    scenario_heat_demands = load_scenario_heat_demands(date_str, period, input_path_hd)
    if not scenario_heat_demands:
        print(f"No scenario heat demands available for date {date_str}.")
        return None, None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None, None

    # Create a DataFrame to store costs
    costs_df = pd.DataFrame({
        't': det_ts_data['t'],
        'deterministic_heat_demand': det_heat_demand
    })

    total_costs = {}
    weighted_total_cost = 0.0

    # For each scenario, compute differences, costs, and total costs
    for scenario_name, scenario_hd_series in scenario_heat_demands.items():
        # Align scenario_hd_series with det_heat_demand
        scenario_hd_series = scenario_hd_series.reindex(det_heat_demand.index)
        scenario_hd_series.name = f"{scenario_name}_heat_demand"

        # Compute difference
        difference = det_heat_demand - scenario_hd_series
        difference.name = f"{scenario_name}_difference"

        # Compute cost per time step
        cost = difference.abs() * 0.01
        cost.name = f"{scenario_name}_cost"

        # Sum costs over time steps
        total_cost = cost.sum()

        # Get probability for the scenario
        probability = probabilities.get(scenario_name, 0)

        # Compute weighted cost
        weighted_cost = total_cost * probability

        # Add to total weighted cost
        weighted_total_cost += weighted_cost

        # Store total cost per scenario
        total_costs[scenario_name] = total_cost

        # Optionally, add to costs_df for detailed output
        costs_df[f"{scenario_name}_cost"] = cost.values

    # Create DataFrame for total costs per scenario
    total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])

    # Display total costs per scenario
    print(f"Total Costs for date {date_str}:")
    display(total_costs_df)

    print(f"Weighted Total Cost for date {date_str}: {weighted_total_cost:.6f}")

    return total_costs_df, weighted_total_cost


In [21]:
def compute_costs_from_differences_special_case(date_str: str, period: str, input_path_det_ts: str, input_path_hd: str):
    """
    Compute the differences and costs between deterministic and scenario heat demands for a specific date,
    including additional costs when heat storage content exceeds a threshold.
    Returns:
        - total_costs_df: DataFrame containing total costs per scenario.
        - weighted_total_cost: The total weighted cost for the date.
    """
    # Load deterministic weighted time series data
    det_ts_data = load_weighted_deterministic_ts(date_str, period, input_path_det_ts)
    if det_ts_data.empty:
        print(f"No deterministic weighted time series data available for date {date_str}.")
        return None, None

    # Extract the deterministic heat demand
    det_heat_demand = det_ts_data['heat_demand']
    det_heat_demand.name = 'deterministic_heat_demand'

    # Extract the deterministic heat storage content
    if 'heat_storage1.heat_capacity' in det_ts_data.columns:
        det_heat_storage_content = det_ts_data['heat_storage1.heat_capacity']
    else:
        print(f"Heat storage content data not available in deterministic time series data for date {date_str}.")
        return None, None
    det_heat_storage_content.name = 'heat_storage1.heat_capacity'

    # Load scenario heat demands
    scenario_heat_demands = load_scenario_heat_demands(date_str, period, input_path_hd)
    if not scenario_heat_demands:
        print(f"No scenario heat demands available for date {date_str}.")
        return None, None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None, None

    # Create a DataFrame to store costs
    costs_df = pd.DataFrame({
        't': det_ts_data['t'],
        'deterministic_heat_demand': det_heat_demand,
        'deterministic_heat_storage_content': det_heat_storage_content,
    })

    total_costs = {}
    weighted_total_cost = 0.0

    # For each scenario, compute differences, costs, and total costs
    for scenario_name, scenario_hd_series in scenario_heat_demands.items():
        # Align scenario_hd_series with det_heat_demand
        scenario_hd_series = scenario_hd_series.reindex(det_heat_demand.index)
        scenario_hd_series.name = f"{scenario_name}_heat_demand"

        # Compute difference
        difference = det_heat_demand - scenario_hd_series
        difference.name = f"{scenario_name}_difference"

        # Compute adjusted storage content
        adjusted_storage_content = det_heat_storage_content + difference
        adjusted_storage_content.name = f"{scenario_name}_adjusted_storage_content"

        # Check if adjusted storage content exceeds 400
        exceeds_capacity = adjusted_storage_content > 400
        additional_cost = exceeds_capacity.astype(float) * 10  # 10 if exceeds, 0 otherwise

        # Compute cost per time step
        cost = difference.abs() * 0.01 + additional_cost
        cost.name = f"{scenario_name}_cost"

        # Sum costs over time steps
        total_cost = cost.sum()

        # Get probability for the scenario
        probability = probabilities.get(scenario_name, 0)

        # Compute weighted cost
        weighted_cost = total_cost * probability

        # Add to total weighted cost
        weighted_total_cost += weighted_cost

        # Store total cost per scenario
        total_costs[scenario_name] = total_cost

        # Optionally, add to costs_df for detailed output
        # costs_df[f"{scenario_name}_cost"] = cost.values

    # Create DataFrame for total costs per scenario
    total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])

    # Display total costs per scenario
    print(f"Total Costs for date {date_str}:")
    display(total_costs_df)

    print(f"Weighted Total Cost for date {date_str}: {weighted_total_cost:.6f}")

    return total_costs_df, weighted_total_cost


In [22]:
# Input paths
input_path_det_ts = INPUT_PATH_D  # Path to deterministic weighted time series
input_path_hd = INPUT_PATH_HD     # Path to heat demand scenarios

# Dictionary to store total costs for each date
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute costs and weighted total cost
    total_costs_df, weighted_total_cost = compute_costs_from_differences_special_case(
        date_str=date_str,
        period=period,
        input_path_det_ts=input_path_det_ts,
        input_path_hd=input_path_hd
    )

    if weighted_total_cost is not None:
        # Load deterministic objective value
        det_obj_value = load_weighted_deterministic_objective_value(date_str, period, INPUT_PATH_D_OBJ)
        if det_obj_value is not None:
            # det_obj_value is a DataFrame with one value
            det_obj_value = det_obj_value['ObjectiveValue'].iloc[0]
            print(f"Deterministic Objective Value for date {date_str}: {det_obj_value}")

            # Compute total cost
            total_cost = det_obj_value + weighted_total_cost

            # Store the result
            results.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Total Cost': total_cost
            })

            print(f"Total Cost for date {date_str}: {total_cost:.6f}")
        else:
            print(f"Could not load deterministic objective value for date {date_str}.")
    else:
        print(f"Could not compute weighted total cost for date {date_str}.")


Processing date: 20230303
Loaded deterministic weighted time series data from ../data/output/deterministic/timeseries/d_weighted_20230303_to_20230303_day_ts.csv


,t,GAS_PRICE,POWER_PRICE,HEAT_PRICE,heat_demand,heat_storage1.initial_soc,chp1.bin,chp1.power,chp1.gas,chp1.heat,...,heat_grid.heat_supply,heat_grid.heat_feedin,arc01_expanded.power,arc02_expanded.power,arc03_expanded.heat,arc04_expanded.heat,arc05_expanded.heat,arc06_expanded.gas,arc07_expanded.gas,arc08_expanded.heat
0,1,0.1543,0.251,0.105,141.657945,320.0,1.0,37.218589,117.039587,68.0,...,1.342055,143.000000,37.218589,38.095238,68.0,75.0,0.000000,0.000000,117.039587,0.000000
1,2,0.1543,0.251,0.105,146.058094,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,146.058094,37.218589,38.095238,68.0,75.0,0.000000,0.000000,117.039587,3.058094
2,3,0.1543,0.251,0.105,234.265871,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,234.265871,37.218589,38.095238,68.0,75.0,91.265871,84.057477,117.039587,0.000000
3,4,0.1543,0.251,0.105,377.818933,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,377.818933,37.218589,38.095238,68.0,75.0,174.000000,162.464986,117.039587,60.818933
4,5,0.1543,0.251,0.105,277.248569,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,277.248569,37.218589,38.095238,68.0,75.0,134.248569,124.792374,117.039587,0.000000


Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Total Costs for date 20230303:


,Scenario,Total Cost
0,Scenario1,3.442595
1,Scenario2,3.784239
2,Scenario3,3.392275
3,Scenario4,4.061540
4,Scenario5,4.209550
5,Scenario6,3.544443
6,Scenario7,3.798941
7,Scenario8,4.023848
8,Scenario9,3.618931
9,Scenario10,2.943745


Weighted Total Cost for date 20230303: 3.616329
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230303_to_20230303_day_obj.csv


...

Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Total Costs for date 20230710:


,Scenario,Total Cost
0,Scenario1,2.852600
1,Scenario2,23.922728
2,Scenario3,3.420707
3,Scenario4,23.605837
4,Scenario5,23.680253
5,Scenario6,13.037037
6,Scenario7,12.914994
7,Scenario8,12.621812
8,Scenario9,3.167353
9,Scenario10,43.460530


Weighted Total Cost for date 20230710: 13.486794
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230710_to_20230710_day_obj.csv


,ObjectiveValue
0,-122.13361


Deterministic Objective Value for date 20230710: -122.13361026377824
Total Cost for date 20230710: -108.646816


In [23]:
# Input paths
input_path_det_ts = INPUT_PATH_D  # Path to deterministic weighted time series
input_path_hd = INPUT_PATH_HD     # Path to heat demand scenarios

# Dictionary to store total costs for each date
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute costs and weighted total cost
    total_costs_df, weighted_total_cost = compute_costs_from_differences(
        date_str=date_str,
        period=period,
        input_path_det_ts=input_path_det_ts,
        input_path_hd=input_path_hd
    )

    if weighted_total_cost is not None:
        # Load deterministic objective value
        det_obj_value = load_weighted_deterministic_objective_value(date_str, period, INPUT_PATH_D_OBJ)
        if det_obj_value is not None:
            # det_obj_value is a DataFrame with one value
            det_obj_value = det_obj_value['ObjectiveValue'].iloc[0]
            print(f"Deterministic Objective Value for date {date_str}: {det_obj_value}")

            # Compute total cost
            total_cost = det_obj_value + weighted_total_cost

            # Store the result
            results.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Total Cost': total_cost
            })

            print(f"Total Cost for date {date_str}: {total_cost:.6f}")
        else:
            print(f"Could not load deterministic objective value for date {date_str}.")
    else:
        print(f"Could not compute weighted total cost for date {date_str}.")


Processing date: 20230303
Loaded deterministic weighted time series data from ../data/output/deterministic/timeseries/d_weighted_20230303_to_20230303_day_ts.csv


,t,GAS_PRICE,POWER_PRICE,HEAT_PRICE,heat_demand,heat_storage1.initial_soc,chp1.bin,chp1.power,chp1.gas,chp1.heat,...,heat_grid.heat_supply,heat_grid.heat_feedin,arc01_expanded.power,arc02_expanded.power,arc03_expanded.heat,arc04_expanded.heat,arc05_expanded.heat,arc06_expanded.gas,arc07_expanded.gas,arc08_expanded.heat
0,1,0.1543,0.251,0.105,141.657945,320.0,1.0,37.218589,117.039587,68.0,...,1.342055,143.000000,37.218589,38.095238,68.0,75.0,0.000000,0.000000,117.039587,0.000000
1,2,0.1543,0.251,0.105,146.058094,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,146.058094,37.218589,38.095238,68.0,75.0,0.000000,0.000000,117.039587,3.058094
2,3,0.1543,0.251,0.105,234.265871,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,234.265871,37.218589,38.095238,68.0,75.0,91.265871,84.057477,117.039587,0.000000
3,4,0.1543,0.251,0.105,377.818933,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,377.818933,37.218589,38.095238,68.0,75.0,174.000000,162.464986,117.039587,60.818933
4,5,0.1543,0.251,0.105,277.248569,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,277.248569,37.218589,38.095238,68.0,75.0,134.248569,124.792374,117.039587,0.000000


Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Total Costs for date 20230303:


,Scenario,Total Cost
0,Scenario1,3.442595
1,Scenario2,3.784239
2,Scenario3,3.392275
3,Scenario4,4.061540
4,Scenario5,4.209550
5,Scenario6,3.544443
6,Scenario7,3.798941
7,Scenario8,4.023848
8,Scenario9,3.618931
9,Scenario10,2.943745


Weighted Total Cost for date 20230303: 3.616329
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230303_to_20230303_day_obj.csv


...

Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Total Costs for date 20230710:


,Scenario,Total Cost
0,Scenario1,2.852600
1,Scenario2,3.922728
2,Scenario3,3.420707
3,Scenario4,3.605837
4,Scenario5,3.680253
5,Scenario6,3.037037
6,Scenario7,2.914994
7,Scenario8,2.621812
8,Scenario9,3.167353
9,Scenario10,3.460530


Weighted Total Cost for date 20230710: 3.224069
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230710_to_20230710_day_obj.csv


,ObjectiveValue
0,-122.13361


Deterministic Objective Value for date 20230710: -122.13361026377824
Total Cost for date 20230710: -118.909542


In [24]:

# Create a DataFrame of results
results_df_eev_special = pd.DataFrame(results)
results_df_eev_special['Total Cost'] = results_df_eev_special['Total Cost'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_eev_special)

# Save the EEV results to a CSV file
filename = "eev_speical.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_eev_special.to_csv(output_filepath, index=False)




Final Results:


,Date,Total Cost
0,2023-03-03,-306.692094
1,2023-03-16,-336.817740
2,2023-04-03,-355.360312
3,2023-04-09,-372.464580
4,2023-05-14,-160.041338
5,2023-05-15,-156.286528
6,2023-06-19,-120.443236
7,2023-06-22,-120.908183
8,2023-06-26,-109.147468
9,2023-07-09,-119.368206


## EEV 

In [25]:

# Create a DataFrame of results
results_df_eev = pd.DataFrame(results)
results_df_eev['Total Cost'] = results_df_eev['Total Cost'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_eev)



Final Results:


,Date,Total Cost
0,2023-03-03,-306.692094
1,2023-03-16,-336.817740
2,2023-04-03,-355.360312
3,2023-04-09,-372.464580
4,2023-05-14,-160.041338
5,2023-05-15,-156.286528
6,2023-06-19,-120.443236
7,2023-06-22,-120.908183
8,2023-06-26,-109.147468
9,2023-07-09,-119.368206


In [26]:
def compute_weighted_stochastic_objective_value(date_str: str, period: str, input_path_s: str, input_path_hd: str) -> float:
    """
    Compute the weighted stochastic objective value for a specific date.
    """
    # Load stochastic objective values
    sto_obj_values_df = load_stochastic_objective_values(date_str, period, input_path_s)
    if sto_obj_values_df.empty:
        print(f"No stochastic objective values data available for date {date_str}.")
        return None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None

    # Clean up column names if necessary
    # Sometimes the first column might have a stray ':' character
    sto_obj_values_df.columns = [col.strip(':') for col in sto_obj_values_df.columns]

    # Merge sto_obj_values_df with probabilities
    sto_obj_values_df['Probability'] = sto_obj_values_df['Scenario'].map(probabilities)

    # Check for missing probabilities
    if sto_obj_values_df['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(sto_obj_values_df[sto_obj_values_df['Probability'].isnull()])
        return None

    # Compute weighted objective value
    sto_obj_values_df['WeightedObjective'] = sto_obj_values_df['ObjectiveValue'] * sto_obj_values_df['Probability']

    # Display intermediate calculations
    print("Intermediate Calculations:")
    display(sto_obj_values_df)

    # Sum the weighted objectives
    weighted_objective_value = sto_obj_values_df['WeightedObjective'].sum()

    return weighted_objective_value


In [27]:
def compute_weighted_stochastic_objective_special_value(date_str: str, period: str, input_path_s: str, input_path_hd: str, special_case: str) -> float:
    """
    Compute the weighted stochastic objective value for a specific date.
    """
    # Load stochastic objective values
    sto_obj_values_df = load_stochastic_objective_special_values(date_str, period, input_path_s, special_case)
    if sto_obj_values_df.empty:
        print(f"No stochastic objective values data available for date {date_str}.")
        return None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None

    # Clean up column names if necessary
    # Sometimes the first column might have a stray ':' character
    sto_obj_values_df.columns = [col.strip(':') for col in sto_obj_values_df.columns]

    # Merge sto_obj_values_df with probabilities
    sto_obj_values_df['Probability'] = sto_obj_values_df['Scenario'].map(probabilities)

    # Check for missing probabilities
    if sto_obj_values_df['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(sto_obj_values_df[sto_obj_values_df['Probability'].isnull()])
        return None

    # Compute weighted objective value
    sto_obj_values_df['WeightedObjective'] = sto_obj_values_df['ObjectiveValue'] * sto_obj_values_df['Probability']

    # Display intermediate calculations
    print("Intermediate Calculations:")
    display(sto_obj_values_df)

    # Sum the weighted objectives
    weighted_objective_value = sto_obj_values_df['WeightedObjective'].sum()

    return weighted_objective_value


In [28]:
# Input paths
input_path_s_obj = INPUT_PATH_S_OBJ  # Path to stochastic objective values
input_path_hd = INPUT_PATH_HD  # Path to heat demand scenarios (probabilities)

# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute weighted stochastic objective value
    weighted_sto_obj_value = compute_weighted_stochastic_objective_value(
        date_str=date_str,
        period=period,
        input_path_s=input_path_s_obj,
        input_path_hd=input_path_hd
    )

    if weighted_sto_obj_value is not None:
        # Store the result
        results.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Weighted Stochastic Objective Value': weighted_sto_obj_value
        })
        print(f"Weighted Stochastic Objective Value for date {date_str}: {weighted_sto_obj_value:.6f}")
    else:
        print(f"Could not compute weighted stochastic objective value for date {date_str}.")


Processing date: 20230303
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230303_to_20230303_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-297.174455,0.213933,-63.575301
1,Scenario2,-297.100131,0.085874,-25.513060
2,Scenario3,-296.415782,0.060216,-17.849005
3,Scenario4,-296.255276,0.115567,-34.237202
4,Scenario5,-296.553264,0.098109,-29.094397
5,Scenario6,-297.363624,0.064584,-19.205043
6,Scenario7,-295.334331,0.078224,-23.102310
7,Scenario8,-296.606585,0.061210,-18.155188
8,Scenario9,-296.174794,0.061475,-18.207455
9,Scenario10,-296.479289,0.160809,-47.676529


Weighted Stochastic Objective Value for date 20230303: -296.615490

Processing date: 20230316
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230316_to_20230316_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-327.439985,0.093211,-30.520853
1,Scenario2,-327.564917,0.041917,-13.730416
2,Scenario3,-326.476060,0.073168,-23.887464
3,Scenario4,-327.798265,0.164225,-53.832764
4,Scenario5,-327.372454,0.096919,-31.728683
5,Scenario6,-327.504890,0.165247,-54.119362
6,Scenario7,-327.215945,0.048252,-15.788962
7,Scenario8,-327.409733,0.114138,-37.369977
8,Scenario9,-327.439659,0.088206,-28.882216
9,Scenario10,-327.372243,0.114716,-37.554953


Weighted Stochastic Objective Value for date 20230316: -327.415649

Processing date: 20230403
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230403_to_20230403_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230403_to_20230403_day.json
Intermediate Calculations:


...

Weighted Stochastic Objective Value for date 20230626: -136.515824

Processing date: 20230709
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230709_to_20230709_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230709_to_20230709_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-139.245134,0.063022,-8.775531
1,Scenario2,-138.214275,0.083171,-11.495435
2,Scenario3,-137.885241,0.133864,-18.457821
3,Scenario4,-137.923714,0.077075,-10.630481
4,Scenario5,-138.863514,0.077908,-10.818560
5,Scenario6,-137.504234,0.121064,-16.646827
6,Scenario7,-137.347280,0.078744,-10.815320
7,Scenario8,-138.968756,0.165071,-22.939698
8,Scenario9,-137.957863,0.145044,-20.009982
9,Scenario10,-138.842718,0.055037,-7.641434


Weighted Stochastic Objective Value for date 20230709: -138.231090

Processing date: 20230710
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230710_to_20230710_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-134.239349,0.157475,-21.139392
1,Scenario2,-133.058427,0.055415,-7.373474
2,Scenario3,-133.931307,0.134182,-17.971196
3,Scenario4,-132.617126,0.115740,-15.349117
4,Scenario5,-132.971352,0.069091,-9.187156
5,Scenario6,-133.356373,0.091832,-12.246385
6,Scenario7,-133.540281,0.110580,-14.766892
7,Scenario8,-133.988860,0.066341,-8.888997
8,Scenario9,-133.551753,0.130086,-17.373206
9,Scenario10,-132.507403,0.069256,-9.176995


Weighted Stochastic Objective Value for date 20230710: -133.472809


In [29]:
# Create a DataFrame of results
results_df_rp = pd.DataFrame(results)
results_df_rp['Weighted Stochastic Objective Value'] = results_df_rp['Weighted Stochastic Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_rp)



Final Results:


,Date,Weighted Stochastic Objective Value
0,2023-03-03,-296.615490
1,2023-03-16,-327.415649
2,2023-04-03,-345.591291
3,2023-04-09,-364.025841
4,2023-05-14,-182.290786
5,2023-05-15,-180.071368
6,2023-06-19,-138.317519
7,2023-06-22,-135.008836
8,2023-06-26,-136.515824
9,2023-07-09,-138.231090


## Special Case RP

In [30]:
# Input paths
input_path_s_obj = INPUT_PATH_S_OBJ  # Path to stochastic objective values
input_path_hd = INPUT_PATH_HD  # Path to heat demand scenarios (probabilities)

# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute weighted stochastic objective value
    weighted_sto_obj_special_value = compute_weighted_stochastic_objective_special_value(
        date_str=date_str,
        period=period,
        input_path_s=input_path_s_obj,
        input_path_hd=input_path_hd,
        special_case=special_case
    )

    if weighted_sto_obj_special_value is not None:
        # Store the result
        results.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Weighted Stochastic Objective Value': weighted_sto_obj_special_value
        })
        print(f"Weighted Stochastic Objective Value for date {date_str} and case {special_case}: {weighted_sto_obj_special_value:.6f}")
    else:
        print(f"Could not compute weighted stochastic objective value for date {date_str}.")


Processing date: 20230303
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230303_to_20230303_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-296.162284,0.213933,-63.358765
1,Scenario2,-296.087960,0.085874,-25.426141
2,Scenario3,-295.403612,0.060216,-17.788056
3,Scenario4,-295.243105,0.115567,-34.120229
4,Scenario5,-295.541093,0.098109,-28.995094
5,Scenario6,-296.351453,0.064584,-19.139672
6,Scenario7,-294.322161,0.078224,-23.023133
7,Scenario8,-295.594414,0.061210,-18.093233
8,Scenario9,-295.162623,0.061475,-18.145232
9,Scenario10,-295.467118,0.160809,-47.513763


Weighted Stochastic Objective Value for date 20230303 and case USE_EXT_COST_10: -295.603319

Processing date: 20230316
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230316_to_20230316_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-327.439985,0.093211,-30.520853
1,Scenario2,-327.564917,0.041917,-13.730416
2,Scenario3,-326.476060,0.073168,-23.887464
3,Scenario4,-327.798265,0.164225,-53.832764
4,Scenario5,-327.372454,0.096919,-31.728683
5,Scenario6,-327.504890,0.165247,-54.119362
6,Scenario7,-327.215945,0.048252,-15.788962
7,Scenario8,-327.409733,0.114138,-37.369977
8,Scenario9,-327.439659,0.088206,-28.882216
9,Scenario10,-327.372243,0.114716,-37.554953


Weighted Stochastic Objective Value for date 20230316 and case USE_EXT_COST_10: -327.415649

Processing date: 20230403
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230403_to_20230403_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230403_to_20230403_day.json
Intermediate Calculations:


...

Weighted Stochastic Objective Value for date 20230626 and case USE_EXT_COST_10: -135.516021

Processing date: 20230709
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230709_to_20230709_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230709_to_20230709_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-138.991712,0.063022,-8.759560
1,Scenario2,-137.960854,0.083171,-11.474358
2,Scenario3,-137.631819,0.133864,-18.423897
3,Scenario4,-137.670293,0.077075,-10.610949
4,Scenario5,-138.610092,0.077908,-10.798817
5,Scenario6,-137.250812,0.121064,-16.616147
6,Scenario7,-137.093859,0.078744,-10.795365
7,Scenario8,-138.715335,0.165071,-22.897866
8,Scenario9,-137.704442,0.145044,-19.973225
9,Scenario10,-138.589296,0.055037,-7.627486


Weighted Stochastic Objective Value for date 20230709 and case USE_EXT_COST_10: -137.977668

Processing date: 20230710
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230710_to_20230710_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-134.162927,0.157475,-21.127358
1,Scenario2,-132.982005,0.055415,-7.369239
2,Scenario3,-133.854886,0.134182,-17.960941
3,Scenario4,-132.540704,0.115740,-15.340272
4,Scenario5,-132.894931,0.069091,-9.181876
5,Scenario6,-123.279952,0.091832,-11.321047
6,Scenario7,-133.463859,0.110580,-14.758442
7,Scenario8,-133.912438,0.066341,-8.883927
8,Scenario9,-133.475331,0.130086,-17.363265
9,Scenario10,-132.430982,0.069256,-9.171702


Weighted Stochastic Objective Value for date 20230710 and case USE_EXT_COST_10: -132.478067


In [31]:
# Create a DataFrame of results
results_df_rp_special = pd.DataFrame(results)
results_df_rp_special['Weighted Stochastic Objective Value'] = results_df_rp_special['Weighted Stochastic Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_rp_special)



Final Results:


,Date,Weighted Stochastic Objective Value
0,2023-03-03,-295.603319
1,2023-03-16,-327.415649
2,2023-04-03,-344.995834
3,2023-04-09,-364.206481
4,2023-05-14,-179.221721
5,2023-05-15,-178.987109
6,2023-06-19,-138.317519
7,2023-06-22,-135.366520
8,2023-06-26,-135.516021
9,2023-07-09,-137.977668


In [32]:
# Save the EEV results to a CSV file
filename = "eev.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_eev.to_csv(output_filepath, index=False)


# Save the WS results to a CSV file
filename = "ws.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df.to_csv(output_filepath, index=False)


# Save the RP results to a CSV file
filename = "rp.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_rp.to_csv(output_filepath, index=False)


In [33]:
# Merge the dataframes on 'Date'
merged_df = pd.merge(results_df, results_df_rp, on='Date')
# Calculate EVPI
merged_df['EVPI'] = merged_df['Weighted Stochastic Objective Value'] - merged_df['Weighted Objective Value']

# Rename columns
merged_df.rename(columns={
    'Weighted Objective Value': 'WS',
    'Weighted Stochastic Objective Value': 'RP'
}, inplace=True)
# Create the results DataFrame with 'Date' and 'EVPI'
results_df_evpi = merged_df[['Date', 'EVPI']]

# Round the values in merged_df to two decimal places
merged_df = merged_df.round(2)

# Display the results
print(results_df_evpi)


          Date       EVPI
0   2023-03-03  12.709231
1   2023-03-16  12.222191
2   2023-04-03  12.923015
3   2023-04-09   6.287475
4   2023-05-14 -18.276978
5   2023-05-15 -20.065364
6   2023-06-19 -21.258376
7   2023-06-22 -21.570263
8   2023-06-26 -24.341929
9   2023-07-09 -13.068072
10  2023-07-10 -10.252959


In [34]:
# Save the EVPI results to a CSV file
filename = "evpi.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_evpi.to_csv(output_filepath, index=False)

# Save the all results to a CSV file
filename = "all.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

merged_df.to_csv(output_filepath, index=False)

In [35]:
def collect_actual_objective_values(dates_to_process: List[datetime], period: str, input_path: str) -> pd.DataFrame:
    """
    Collect the actual deterministic objective values for a list of dates.

    Parameters:
    - dates_to_process: List of datetime objects representing the dates to process.
    - period: The period string used in the filenames (e.g., 'day', 'week').
    - input_path: The path where the objective value CSV files are located.

    Returns:
    - A pandas DataFrame with columns 'Date' and 'Actual Value'.
    """
    import os
    import pandas as pd
    from datetime import datetime

    actual_values = []

    for date in dates_to_process:
        date_str = date.strftime('%Y%m%d')
        # Use your existing function to load the data
        data = load_actual_deterministic_objective_values(date_str, period, input_path)
        if not data.empty:
            # Assuming the CSV has columns 'Scenario' and 'ObjectiveValue'
            actual_row = data[data['Scenario'] == 'actual']
            if not actual_row.empty:
                objective_value = actual_row['ObjectiveValue'].iloc[0]
                actual_values.append({
                    'Date': date.strftime('%Y-%m-%d'),
                    'Actual Value': objective_value
                })
            else:
                print(f"No 'actual' scenario found in data for date {date_str}")
        else:
            print(f"Data not available for date {date_str}")

    # Create a DataFrame from the results
    actual_values_df = pd.DataFrame(actual_values)
    # Sort by date
    actual_values_df = actual_values_df.sort_values('Date').reset_index(drop=True)
    return actual_values_df


In [36]:
input_path = INPUT_PATH_D_OBJ_ACT  # Path to actual deterministic objective values

actual_values_df = collect_actual_objective_values(dates_to_process, period, input_path)

# Display the DataFrame
print("\nActual Objective Values:")
print(actual_values_df)

# Optionally, write the DataFrame to a CSV file
filename = "actual_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
actual_values_df.to_csv(output_filepath, index=False)
print(f"\nActual objective values have been written to {output_filepath}")

Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230303_to_20230303_day_obj.csv


Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230316_to_20230316_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230403_to_20230403_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230409_to_20230409_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230514_to_20230514_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230515_to_20230515_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230619_to_20230619_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230622_to_20230622_day_obj.csv
Loaded actual deterministic objective values from ../data/out

In [37]:
def collect_objective_values(dates_to_process: List[datetime], period: str, input_path: str) -> pd.DataFrame:
    """
    Collect the actual deterministic objective values for a list of dates.

    Parameters:
    - dates_to_process: List of datetime objects representing the dates to process.
    - period: The period string used in the filenames (e.g., 'day', 'week').
    - input_path: The path where the objective value CSV files are located.

    Returns:
    - A pandas DataFrame with columns 'Date' and 'Actual Value'.
    """
    import os
    import pandas as pd
    from datetime import datetime

    values = []

    for date in dates_to_process:
        date_str = date.strftime('%Y%m%d')
        # Use your existing function to load the data
        data = load_deterministic_objective_values(date_str, period, input_path)
        if not data.empty:
            if 'ObjectiveValue' in data.columns:
                objective_value = data['ObjectiveValue'].iloc[0]
                values.append({
                    'Date': date.strftime('%Y-%m-%d'),
                    'Objective Value': objective_value
                })
            else:
                print(f"Die Spalte 'ObjectiveValue' wurde in der Datei {filename} nicht gefunden.")
        else:
            print(f"Datei {filename} existiert nicht im Pfad {input_path}.")
            
            
           
    # Create a DataFrame from the results
    values_df = pd.DataFrame(values)
    # Sort by date
    values_df = values_df.sort_values('Date').reset_index(drop=True)
    return values_df


In [38]:
input_path = INPUT_PATH_D_OBJ  # Ersetzen Sie dies durch den tatsächlichen Pfad

# Aufrufen der Funktion
objective_values_df = collect_objective_values(dates_to_process, period, input_path)

# Anzeigen der Ergebnisse
print(objective_values_df)

# Optionally, write the DataFrame to a CSV file
filename = "deterministic_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
objective_values_df.to_csv(output_filepath, index=False)
print(f"\nObjective values have been written to {output_filepath}")

Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230303_to_20230303_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230316_to_20230316_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230403_to_20230403_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230409_to_20230409_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230514_to_20230514_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230515_to_20230515_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230619_to_20230619_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230622_to_20230622_day_obj.csv


In [39]:
# Merge the first two DataFrames on 'Date'
merged_df = pd.merge(objective_values_df, results_df_rp, on='Date', how='outer')
# Merge the result with the third DataFrame
merged_df = pd.merge(merged_df, actual_values_df, on='Date', how='outer')

# Rename columns
merged_df.rename(columns={
    'Objective Value': 'DM Objective Value',
    'Weighted Stochastic Objective Value': 'SM Objective Value',
    'Actual Value': 'Actual Objective Value'
}, inplace=True)

merged_df = merged_df.round(2)

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df)

# Optionally, write the DataFrame to a CSV file
filename = "all_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
merged_df.to_csv(output_filepath, index=False)
print(f"\nObjective values have been written to {output_filepath}")

Merged DataFrame:
          Date  DM Objective Value  SM Objective Value  Actual Objective Value
0   2023-03-03             -301.49             -296.62                 -291.04
1   2023-03-16             -331.87             -327.42                 -343.49
2   2023-04-03             -350.38             -345.59                 -362.75
3   2023-04-09             -369.07             -364.03                 -298.54
4   2023-05-14             -188.67             -182.29                 -193.84
5   2023-05-15             -187.04             -180.07                 -187.42
6   2023-06-19             -143.00             -138.32                 -134.83
7   2023-06-22             -139.96             -135.01                 -124.88
8   2023-06-26             -141.88             -136.52                 -132.99
9   2023-07-09             -144.07             -138.23                 -130.27
10  2023-07-10             -138.54             -133.47                 -124.21

Objective values have been writte